In [ ]:
!pip install lmfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 11.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import lmfit

from scipy.stats import norm
from scipy.interpolate import splrep, splev
from scipy.optimize import minimize, fsolve

from astropy.io import fits

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)
plt.rc('xtick', labelsize=medium_size)
plt.rc('ytick', labelsize=medium_size)
plt.rc('legend', fontsize=medium_size)
plt.rc('axes', titlesize=large_size)
plt.rc('axes', labelsize=large_size)
plt.rc('figure', titlesize=large_size)

Read Image FITS Files

In [ ]:
def read_image(filename,flip=True):
    # Open file
    file = fits.open(filename)

    # Read header and image data
    header = file[0].header
    image = file[0].data

    # Flipping the image if needed
    if flip:
        image = np.flip(image,axis=0)

    print(image.shape)
    print(image.dtype)

    return header,image

Background Extraction

In [ ]:
def remove_background(image,bins=51,show_fit=False,show_graph=False,debug_mode=False):
    '''
    Fitting gaussian noise from signal histogram and removing background signal.

    Input:
        image           : Image array from fits file
        bins            : Number of bin edges for histogram
        show_fit        : Show fitting statistics
        show_graph      : Visualize histogram and fits

    Output:
        cleaned_image   : Image array with background removed
        background      : Mean background signal
        sigma           : Standard deviation of background signal
    '''

    # Region mostlikely contain background signal
    temp_image = image[np.where(np.logical_and(image>image.mean()-3*image.std(),
                                               image<image.mean()+3*image.std()))].flatten()

    bins = np.linspace(max(temp_image.mean()-3*temp_image.std(),0),
                           temp_image.mean()+3*temp_image.std(),bins)

    # Signal histogram
    hist,bins = hist,ibins = np.histogram(image.flatten(),bins=bins)
    bin_centers = (bins[1:]+bins[:-1])/2

    # Normalizing histogram
    scalling_factor = (bin_centers.shape[0])/np.sum(hist)/(bins[-1]-bins[0])
    hist, hist_err = hist*scalling_factor, np.sqrt(hist)*scalling_factor

    if debug_mode:
        plt.figure()
        plt.scatter(bin_centers,hist)
        plt.show()

    # Background fit
    def gaussian_noise(x,A,x0,sigma):
        return A*norm.pdf(x,loc=x0,scale=sigma)

    # Masking data. We only fit the leftside of the gaussian peak.
    noise_mask = hist != 0

    # Using lmfit to fit the gaussian noise function
    model = lmfit.Model(gaussian_noise)
    params = model.make_params(
        A = 1,
        x0 = bin_centers[np.argmax(hist)],
        sigma = np.std(image[np.where(np.logical_and(image>bins[0],image<bins[-1]))].flatten())
    )
    result = model.fit(
        data = hist[noise_mask],
        params = params,
        x = bin_centers[noise_mask],
        weights = 1/hist_err[noise_mask]
    )
    scale,noise_peak,sigma = result.params['A'].value,result.params['x0'].value,result.params['sigma'].value

    if show_fit:
        lmfit.report_fit(result)

    # Plotting
    if show_graph:
        bin_centers_highreg = np.linspace(bin_centers[0],bin_centers[-1],20*bin_centers.shape[0])

        _,ax = plt.subplots(2,1,figsize=(9,12))

        # Full Signal
        # Setting up ticks (for pretty plots)
        ax[0].tick_params(axis='both', which='both',direction='in',top=True,right=True)
        ax[0].tick_params(axis='both', which='major',length=6)

        # Plotting uncut signal histogram
        ax[0].errorbar(bin_centers,hist,hist_err,
                       color='steelblue',fmt=' ',elinewidth=2,capsize=5,
                       label='Signal Histogram')

        # Visualizing noises
        # Gaussian noise functions
        ax[0].plot(bin_centers_highreg,scale*norm.pdf(bin_centers_highreg,loc=noise_peak,scale=sigma),
                   color='orangered',
                   label='Gaussian Fit')
        # Noise peaks
        ax[0].axvline(noise_peak,
                      linestyle='-.',alpha=0.3,color='darkgreen',
                      label='Background Peak')
        # 5-sigma regions
        ax[0].axvspan(noise_peak-5*sigma,noise_peak+5*sigma,
                      linestyle='--',alpha=0.3,color='cadetblue',
                      label=r'5$\sigma$ Region')

        # Naming things
        ax[0].set_xlabel('Pixel Value')
        ax[0].set_ylabel('Histogram')

        ax[0].set_ylim(bottom=0)
        ax[0].set_xlim(noise_peak-8*sigma,noise_peak+8*sigma)

        ax[0].legend()

        # De-noise Signal
        # Setting up ticks (for pretty plots)
        ax[1].tick_params(axis='both', which='both',direction='in',top=True,right=True)
        ax[1].tick_params(axis='both', which='major',length=6)

        # Plotting uncut signal histogram
        ax[1].errorbar(bin_centers,hist,hist_err,
                       color='darkgreen',fmt=' ',elinewidth=2,capsize=5,
                       label='Signal Histogram')
        # Plotting signal histogram without gaussian noise
        ax[1].errorbar(bin_centers,hist-scale*norm.pdf(bin_centers,loc=noise_peak,scale=sigma),hist_err,
                       color='steelblue',fmt=' ',elinewidth=2,capsize=5,
                       label='De-noised Histogram')

        # Plotting noise peaks and 5-sigma regions
        # Noise peaks
        ax[1].axvline(noise_peak,
                      linestyle='-.',alpha=0.3,color='darkgreen',
                      label='Background Peak')
        # 5-sigma regions
        ax[1].axvspan(noise_peak-5*sigma,noise_peak+5*sigma,
                      linestyle='--',alpha=0.3,color='cadetblue',
                      label=r'5$\sigma$ Region')

        # Naming things
        ax[1].set_xlabel('Pixel Value')
        ax[1].set_ylabel('Histogram')

        ax[1].set_ylim(bottom=0)
        ax[1].set_xlim(noise_peak-8*sigma,noise_peak+8*sigma)

        ax[1].legend()

        plt.show()

    return image-noise_peak, noise_peak, sigma

Objects Search

In [ ]:
def signal_indices(cleaned_image,sigma):
    '''
    Choosing pixels with signal > 5*sigma

    Input:
        cleaned_image   : De-noise image, first output of remove_background
        sigma           : Gaussian noise (background) standard deviation, third output of remove_background

    Output:
        signal_indices  : Set of signal coordinates (y,x indices)
    '''

    ys,xs = np.where(cleaned_image > 5*sigma)

    return set((iy,ix) for iy,ix in zip(ys,xs))

def object_search(all_signal_indices):
    '''
    Utilizing flood (graph) search to catalog objects
    Objects are defined as sets of pixel adjoining each other

    Input:
        all_signal_indices  : All signal indices, signal_indices output

    Output:
        group_list          : List of objects, objects are stored in sets of pixels indices (y,x coordinate)
    '''

    # Setting up group list and seen pixels set
    group_list = []
    seen = set()

    # Agenda for looping over pixels in all_signal_indices
    agenda = list(all_signal_indices)
    i = 0

    # Looping over pixels in agenda
    while i < len(agenda):
        starting_pixel = agenda[i]
        i += 1

        # Setting up current group and sub-agenda for flood searching
        current_group = set()
        sub_agenda = [starting_pixel]

        # Flood search
        while sub_agenda:
            # Remove the last pixel from sub_agenda
            current_pixel = sub_agenda.pop(-1)

            # Check if the code has seen the current pixel or not
            if current_pixel in seen:
                continue

            # If not seen, add the current pixel to the seen set
            seen.add(current_pixel)

            if current_pixel in all_signal_indices:
                # If current pixel is a signal indices, add it to the current group
                current_group.add(current_pixel)

                # Adding current pixcel's adjacent pixels to agenda
                current_y, current_x = current_pixel
                current_neightboors = [
                    (current_y+1,current_x),
                    (current_y-1,current_x),
                    (current_y,current_x+1),
                    (current_y,current_x-1)
                ]

                sub_agenda += current_neightboors

        # If current group is not empty, add it to the group list
        if current_group:
            group_list.append(current_group)

    return group_list


Remove Hot-pixels

In [ ]:
def hot_and_warm_pixels_cut_choise(group_list,largest_cut=73):
    '''
    Examinize hot pixel cuts by changing the cutting criteria
    and plotting the number of rejected and accepted objects per criteria

    Input:
        group_list      : List of objects, output of object_search
        largest_cut     : The largest cutting criteria that will be used

    Output:
        None - Only generate plot
    '''

    # Cutting criteria
    cuts = np.arange(1,largest_cut,2)+1
    nobjects = np.array([])

    # Finding the number of accepted and rejected objects per cutting criterion
    for cut in cuts:
        nobjects = np.append(nobjects,np.sum([1 for group in group_list if len(group) < cut]))

    # Plotting
    # Using ax1 (left y axis) for plotting numbers of rejected groups and
    # the twin ax2 (right y axis) for plotting numbers of accepted groups
    _,ax1 = plt.subplots()

    # Setting up ticks for ax1(for pretty plots)
    ax1.tick_params(axis='both', which='both',direction='in',top=True,right=True)
    ax1.tick_params(axis='both', which='major',length=6)

    # Creating ip ax2
    ax2 = ax1.twinx()

    # Setting up ticks for ax2(for pretty plots)
    ax2.tick_params(axis='both', which='both',direction='in',top=True,right=True)
    ax2.tick_params(axis='both', which='major',length=6)

    # Plotting numbers of rejected objects
    ax1.errorbar(cuts,nobjects,
                 np.sqrt(nobjects),
                 color='steelblue',fmt=' ',
                 elinewidth=2,capsize=5,
                 label='Rejected Objects')

    # Plotting numbers of accepted objects
    ax2.errorbar(cuts,np.ones_like(nobjects)*len(group_list)-nobjects,
                 np.sqrt(np.ones_like(nobjects)*len(group_list)-nobjects),
                 color='orangered',fmt=' ',
                 elinewidth=2,capsize=5,
                 label='Accepted Objects')

    # Naming things
    ax1.set_xlabel('Cut Size [Pixel]')
    ax1.set_ylabel('Number of Rejected Object')
    ax2.set_ylabel('Number of Accepted Object')

    ax1.legend(loc='lower left')
    ax2.legend(loc='upper right')
    plt.show()

def remove_hot_pixels_and_extended_objects(image,group_list,sigma,limiting_size=[30,np.pi*30**2]):
    '''
    Removing hot pixels and replacing them with randomly sampled value from the gaussian noise,
    as well as creating a separated object list for extended objects
    !!! This may also remove smaller stars, which is intentional for the purpose of more accurately calculate star radius

    Input:
        image               : De-noise image array, first output of remove_background
        group_list          : List of objects, output of object_search
        sigma               : Gaussian noise (background) standard deviation, third output of remove_background
        limiting_size       : Consisting of two elements, the first is the cut for hot pixel, the second is the cut for extended objects

    Output:
        cleaned_image       : De-noise and hot-pixels-removed image array
        cleaned_star_list   : List of objects without hot pixels and extended objects
        extended_object_list: List of extended objects
    '''

    # Setting up list
    cleaned_star_list = []
    extended_object_list = []

    # Making a clone of the de-noise image array
    cleaned_image = image + 0 # The + 0 is neccessary for us to keep the initial de-noise image unchanged

    # Looping through groups in group list
    for group in group_list:
        # Rooting out hot pixels (and smaller stars) and replacing them with sampled values from the noise (background) gaussian distribution
        if len(group) < limiting_size[0]:
            for y,x in group:
                cleaned_image[y,x] = np.random.normal(scale=sigma)

        # Adding accepted objects to cleaned star list
        elif len(group) < limiting_size[1]:
            cleaned_star_list.append(group)

        # Adding the rest of the objects into extended object list
        else:
            extended_object_list.append(group)
    return cleaned_image,cleaned_star_list,extended_object_list

Stars Analysis

In [ ]:
def star_analysis(image,group,noise_sigma,psf_or_gaussian='gaussian',
                  focus=True,saturated_level=2**16,
                  extend=0,show_fit=False):
    '''
    From the groups' data, calculate stars' parameters - star's center, cut-out ccd image array,
    zero index(the top left pixel index), whether or not the star is saturated, pixels' distances (radii)
    from the star's center and those pixels intensity, as well as star's radius for point source, circle inner
    and outer radius for de-focus photometry.

    Input:
        image           : De-noise and hot-pixels-removed image array, first output of remove_hot_pixels_and_extended_objects
        group           : List of objects without hot pixels and extended objects, second output of remove_hot_pixels_and_extended_objects
        noise_sigma     : Gaussian noise (background) standard deviation, third output of remove_background
        psf_or_gaussian : Type of fitting function for point source stars
        focus           : Type of photometry
        saturated_level : Saturated level of ccd (remember to reduce the origional saturated level by the background level)
        extend          : Extention of star cut-out ccd image array (extention on each of the 4 directions)
        show_fit        : Show the radius fitting

    Output:
        star            : A star instance, containing all the information calculated
    '''

    # Initialize a star instance
    star = {}

    # List out the coordinates of pixels as two separated y and x arrays
    ys,xs = [],[]
    for iy,ix in group:
        ys.append(iy)
        xs.append(ix)

    # Storing the cut-out ccd image array, zero index, and saturated state of the star
    star['ccd'] = image[min(ys)-extend:max(ys)+1+extend,
                        min(xs)-extend:max(xs)+1+extend]
    star['zero_index'] = (min(ys)-extend,min(xs)-extend)
    star['saturated'] = np.max(star['ccd']) < saturated_level

    # Calculating the center of the star by minimizing the weighted distances
    # of each pixel to the center using the pixels' intensities as weights

    # Guessing a temporary center
    center_guess = (np.mean(ys),np.mean(xs))

    # Function that need to be minimized
    def circle_loss_func(x):
        yc,xc = x
        return np.sum([
            image[iy,ix]**2 * ((yc-iy)**2 + (xc-ix)**2) for iy,ix in group
        ])
    # Minimizing the function to find the star's center
    star['center'] = tuple(minimize(circle_loss_func,center_guess).x)

    # From the star's center, calculate pixels' radii and create a coresponding array for the intensities
    star['radii'] = np.array([
        np.sqrt((star['center'][0]-iy)**2 + (star['center'][1]-ix)**2) for iy,ix in group
    ])
    star['pixel_val'] = np.array([
        image[iy,ix] for iy,ix in group
    ])

    # Finding star's radius for point source, circle inner and outer radius for de-focus photometry
    # For point source
    if focus:
        # Mirroring the star's pixels radii and pixel values (intensities) for better fitting
        extended_radii = np.append(star['radii'],-star['radii'])
        extended_pixel_val = np.append(star['pixel_val'],star['pixel_val'])

        # Defind psf and gaussian model for fitting
        def psf_approx(x,A,scale):
            mod_scale = abs(scale) + 0.1
            return np.where(x!=0,A*np.power( np.sin(x/mod_scale)/(x/mod_scale) ,2),A)

        def gaussian_approx(x,A,scale):
            return A*norm.pdf(x,scale=scale)

        # Choosing function to fit
        approx_func = psf_approx if psf_or_gaussian == 'psf' else gaussian_approx

        # Fitting using lmfit
        model = lmfit.Model(approx_func)
        params = model.make_params(
            A = np.max(star['pixel_val']),
            scale = np.sqrt(np.sum((star['radii']*star['pixel_val']/np.sum(star['pixel_val']))**2))
        )
        result = model.fit(
            data = extended_pixel_val,
            params = params,
            x = extended_radii,
            weights = 1/np.sqrt(extended_pixel_val)/extended_radii
        )

        # Fitting result
        A = result.params['A'].value
        scale = result.params['scale'].value

        # Find the star's radius (half-width at half maximum)
        def find_star_radius(x):
            return approx_func(x,A,scale) - approx_func(0,A,scale)/2

        star['radius'] = fsolve(find_star_radius,scale)[0]

        # Showing the fit
        if show_fit:
            # Printing fit statistic
            lmfit.report_fit(result)

            # Initialize plots
            _,ax = plt.subplots(2,1,figsize=(9,12))

            # Plotting point-spread function without fit
            # Setting up ticks (for pretty plots)
            ax[0].tick_params(axis='both', which='both',direction='in',top=True,right=True)
            ax[0].tick_params(axis='both', which='major',length=6)

            # Plotting the pixels' values vs. pixels' distances (radii) from star's center
            ax[0].errorbar(np.append(star['radii'],-star['radii']),
                           np.append(star['pixel_val'],star['pixel_val']),
                           np.sqrt(np.append(star['pixel_val'],star['pixel_val'])),
                           color='steelblue',fmt=' ',
                           elinewidth=2,capsize=5,
                           label='Point-spread Function'
                           )

            # Showing the 5-sigma region
            ax[0].axhspan(-5*noise_sigma,5*noise_sigma,
                          linestyle='-.',alpha=0.3,color='cadetblue',
                          label = '5-sigma Noise Cut')

            # Naming things
            ax[0].set_xlabel('Radius [Pixel]')
            ax[0].set_ylabel('Pixel Value')

            ax[0].set_ylim(bottom=0)

            ax[0].legend()

            # Plotting point-spread function with fit
            # Setting up ticks (for pretty plots)
            ax[1].tick_params(axis='both', which='both',direction='in',top=True,right=True)
            ax[1].tick_params(axis='both', which='major',length=6)

            # Plotting the pixels' values vs. pixels' distances (radii) from star's center
            ax[1].errorbar(np.append(star['radii'],-star['radii']),
                           np.append(star['pixel_val'],star['pixel_val']),
                           np.sqrt(np.append(star['pixel_val'],star['pixel_val'])),
                           color='steelblue',fmt=' ',
                           elinewidth=2,capsize=5,
                           label='Point-spread Function'
                           )

            # Plotting the fitted pixels' values vs. pixels' distances (radii) from star's center
            fit_radii = np.linspace(-5*star['radius'],5*star['radius'],1000)
            ax[1].plot(fit_radii,approx_func(fit_radii,A,scale),
                       color='orangered',label='PSF Fit')

            # Showing the 5-sigma region
            ax[1].axhspan(-5*noise_sigma,5*noise_sigma,
                          linestyle='-.',alpha=0.3,color='cadetblue',
                          label = '5-sigma Noise Cut')
            # Showing the star region (within the star's radius)
            ax[1].axvspan(-star['radius'],star['radius'],
                          linestyle='--',alpha=0.3,color='darkgreen',
                          label = 'Star Region')

            # Naming things
            ax[1].set_xlabel('Radius [Pixel]')
            ax[1].set_ylabel('Pixel Value')

            ax[1].set_ylim(bottom=0)
            ax[1].set_xlim(-5*star['radius'],5*star['radius'])

            ax[1].legend()

            plt.show()
    # For circle (un-focus mode)
    else:
        # Calculating pixels' radii and create a coresponding array for the intensities for all pixels in the extended image array
        full_radii = np.array([
            np.sqrt((star['center'][0]-star['zero_index'][0]-iy)**2 + (star['center'][1]-star['zero_index'][1]-ix)**2) for iy in range(star['ccd'].shape[0]) for ix in range(star['ccd'].shape[1])
        ])
        full_pixel_val = np.array([
            star['ccd'][iy,ix] for iy in range(star['ccd'].shape[0]) for ix in range(star['ccd'].shape[1])
        ])

        # Define circle model for fitting
        def de_focus_approx(x,A,bg,inner_r,outer_r):
            return np.where(np.logical_and(x>inner_r,x<outer_r),A,bg)

        # Fitting using lmfit
        model = lmfit.Model(de_focus_approx)
        params = model.make_params(
            A = np.max(star['pixel_val']),
            bg = np.min(star['pixel_val']),
            inner_r = np.min(star['radii']),
            outer_r = np.max(star['radii'])
        )
        result = model.fit(
            data = full_pixel_val,
            params = params,
            x = full_radii,
            wweights = 1/np.sqrt(full_pixel_val)/full_radii
        )

        # Fitting result
        A = result.params['A'].value
        bg = result.params['bg'].value

        # Finding the star circle's inner and outer radii
        star['inner_radius'],star['outer_radius'] = result.params['inner_r'].value,result.params['outer_r'].value

        # Showing the fit
        if show_fit:
            # Printing fit statistic
            lmfit.report_fit(result)

            # Initialize plots
            _,ax = plt.subplots(2,1,figsize=(9,12))

            # Plotting point-spread function without fit
            # Setting up ticks (for pretty plots)
            ax[0].tick_params(axis='both', which='both',direction='in',top=True,right=True)
            ax[0].tick_params(axis='both', which='major',length=6)

            # Plotting the pixels' values vs. pixels' distances (radii) from star's center
            ax[0].errorbar(star['radii'],star['pixel_val'],
                           np.sqrt(star['pixel_val']),
                           color='steelblue',fmt=' ',
                           elinewidth=2,capsize=5,
                           label='Point-spread Function')

            # Showing the 5-sigma region
            ax[0].axhspan(-5*noise_sigma,5*noise_sigma,
                          linestyle='-.',alpha=0.3,color='cadetblue',
                          label = '5-sigma Noise Cut')

            # Naming things
            ax[0].set_xlabel('Radius [Pixel]')
            ax[0].set_ylabel('Pixel Value')

            ax[0].ylim(bottom=0)

            ax[0].legend()

            # Plotting point-spread function with fit
            # Setting up ticks (for pretty plots)
            ax[1].tick_params(axis='both', which='both',direction='in',top=True,right=True)
            ax[1].tick_params(axis='both', which='major',length=6)

            # Plotting the pixels' values vs. pixels' distances (radii) from star's center
            ax[1].errorbar(star['radii'],star['pixel_val'],
                           np.sqrt(star['pixel_val']),
                           color='steelblue',fmt=' ',
                           elinewidth=2,capsize=5,
                           label='Point-spread Function')

            # Plotting the fitted pixels' values vs. pixels' distances (radii) from star's center
            ax[1].plot(full_radii,de_focus_approx(full_radii,A,bg,star['inner_radius'],star['outer_radius']),
                       color='orangered',label='De-focus Fit')

            # Showing the 5-sigma region
            ax[1].axhspan(-5*noise_sigma,5*noise_sigma,
                          linestyle='-.',alpha=0.3,color='cadetblue',
                          label = '5-sigma Noise Cut')
            # Showing the star region (between the star circle's inner and outer radii)
            ax[1].axvspan(star['inner_radius'],star['outer_radius'],
                          linestyle='--',alpha=0.3,color='darkgreen',
                          label = 'Star Region')

            # Naming things
            ax[1].set_xlabel('Radius [Pixel]')
            ax[1].set_ylabel('Pixel Value')

            ax[1].ylim(bottom=0)
            ax[1].xlim(0,1.5*star['outer_radius'])

            ax[0].legend()

            plt.show()

    return star

Clean Star Info

In [ ]:
def clean_stars_data_psf(stars_info,bins=41,show_fit=False,debug_mode=False):
    '''
    From the stars' radii (inner and outer radii in the case of de-focus mode),
    calculate their mean and standard deviation to use for photometry.

    Input:
        stars_info  : A list of star instances calculated from star_analysis
        bins        : Number of bin edges for the radius histogram
        show_fit    : Visualize the radius gaussian fit

    Output:
        None - Only change the stars' information
    '''

    # Creating a radius array for easier analysis
    stars_radii = np.array([
        star['radius'] for star in stars_info if not np.isnan(star['radius'])
    ])

    # Finding the mean radius and standard deviation from the radius array;
    # note that this is not necessary the same as the gaussian model mean and standard deviation
    stars_radii = stars_radii[np.where(np.logical_and(stars_radii > 0,
                                                      stars_radii < 10))]

    mean_radius = stars_radii.mean()
    sigma_radius = stars_radii.std()

    bins = np.linspace(max(0,mean_radius-5*sigma_radius),
                       min(mean_radius+5*sigma_radius,2*mean_radius),bins)

    # Creating radius histogram
    radius_hist,bins = np.histogram(stars_radii,bins=bins)
    radius_hist_err = np.sqrt(radius_hist)
    bin_centers = (bins[:-1]+bins[1:])/2

    # Normalizing the histogram
    scalling = bin_centers.shape[0]/(bins[-1]-bins[0])/np.sum(radius_hist)

    if debug_mode:
        plt.figure()
        plt.scatter(bin_centers,radius_hist)
        plt.show()

    # Masking histogram data for fitting
    mask = (radius_hist != 0)
    bin_centers, radius_hist, radius_hist_err = bin_centers[mask], radius_hist[mask]*scalling, radius_hist_err[mask]*scalling

    # Fitting the gaussian model with lmfit
    def gaussian_fit(x,mean,sigma):
        return norm.pdf(x,loc=mean,scale=sigma)

    model = lmfit.Model(gaussian_fit)
    params = model.make_params(
        mean = np.sum(bin_centers*radius_hist*(bins[1]-bins[0])),
        sigma = sigma_radius,
    )
    result = model.fit(
        data = radius_hist,
        params = params,
        x = bin_centers,
        weights = 1/radius_hist_err
    )

    # The resulted mean radius and standard deviation
    mean_radius, sigma_radius = result.params['mean'].value, result.params['sigma'].value

    # Visualizing firt
    if show_fit:
        # Printing out the fitted parameters
        print('Mean Radius: ',mean_radius)
        print('Sigma: ',sigma_radius)

        lmfit.report_fit(result)

        # Initializing plot
        _,ax = plt.subplots()

        # Setting up ticks (for pretty plots)
        ax.tick_params(axis='both', which='both',direction='in',top=True,right=True)
        ax.tick_params(axis='both', which='major',length=6)

        # Plotting the radius histogram
        plt.errorbar(bin_centers,radius_hist,radius_hist_err,
                     color='steelblue',fmt=' ',elinewidth=2,capsize=5,
                     label="Stars' Radii")

        # Plotting the gaussian fit
        gaussian_radii = np.linspace(bin_centers[0],bin_centers[-1],100)
        plt.plot(gaussian_radii,gaussian_fit(gaussian_radii,mean_radius,sigma_radius),
                 color='orangered',label='Gaussian Fit')

        # Naming thinsg
        plt.xlabel('Radius [Pixel]')
        plt.ylabel('Histogram')

        plt.legend()
        plt.show()

    # Replacing each star's radius into the newly calculated mean radius and standard deviaiton
    for star in stars_info:
        star['radius'] = [mean_radius,sigma_radius]

Photometry

In [ ]:
def photometry(stars_info,focus=True,inter=1000):
    '''
    Photometry process using Monte Carlo simulation (because why not)

    Input:
        stars_info  : A list of star instances calculated from star_analysis and processed through clean_stars_data_psf
        focus       : Photometry mode
        inter       : Number of interations with Monte Carlo simulation

    Output:
        None - Only calculate and add photometry intensities into star instances
    '''

    # Monte Carlo simulation
    def monte_carlo_flux(star,flux_func):
        fluxes = np.array([])
        for i in range(inter):
            fluxes = np.append(fluxes,flux_func(star))
        return fluxes.mean(), np.sqrt(fluxes.mean()+fluxes.var())

    # Photometry for point source with randomly sampled radius (following a gaussian distribution)
    if focus:
        def flux_focus_func(star):
            radius = np.random.normal(loc=star['radius'][0],scale=star['radius'][1])
            return np.sum(star['pixel_val'][star['radii'] < radius])

        for star in stars_info:
            star['flux'], star['flux_err'] = monte_carlo_flux(star,flux_focus_func)
    # Photometry for de-focus mode with randomly sampled inner and outer radii (following gaussian distributions)
    else:
        def flux_unfocus_func(star):
            inner_radius = np.random.normal(loc=star['inner_radius'][0],scale=star['inner_radius'][1])
            outer_radius = np.random.normal(loc=star['outer_radius'][0],scale=star['outer_radius'][1])
            return np.sum(star['pixel_val'][np.where(np.logical_and(star['radii'] > inner_radius,star['radii'] < outer_radius))])

        for star in stars_info:
            star['flux'], star['flux_err'] = monte_carlo_flux(star,flux_unfocus_func)

Saving Photometry Data

In [ ]:
def save_stars_info(stars_info,file_name,fields_name=['flux','flux_err']):
    '''
    Saving the processed data into a csv file

    Input:
        stars_info      : A list of star instances calculated from star_analysis, processed through clean_stars_data_psf and photometry
        file_name       : The file path for the csv file
        fields_names    : Fields to save

     Output:
        None - Only save the csv file
    '''
    with open(file_name,'w') as f:
        writer = csv.writer(f)

        writer.writerow(['y','x'] + fields_name)

        for star in stars_info:
            writer.writerow([star['center'][0], star['center'][1]] + [
                star[name] for name in fields_name
            ])

TESTING